# Prepare Dataset

In [1]:
import os
#hfhome_dir="/data/cmpe249-fa23/Huggingfacecache"
hfhome_dir=os.path.join('D:',os.sep, 'Cache','huggingface')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
#os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1

In [3]:
from datasets import load_dataset, load_metric
# dataset_split = load_dataset(
#         "librispeech_asr", #'librispeech_asr'
#         'clean',
#         split='train.100',
#     )
data_dir=r"D:\GitHub\datasets\timit\TIMIT"
dataset_split = load_dataset("timit_asr", data_dir=data_dir)

Generating train split: 4620 examples [00:45, 100.98 examples/s]
Generating test split: 1680 examples [00:16, 100.76 examples/s]


In [4]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [9]:
dataset_iter = iter(dataset_split['train'])
sample = next(dataset_iter)

In [10]:
sample

{'file': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
 'audio': {'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
  'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
         -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]),
  'sampling_rate': 16000},
 'text': 'She had your dark suit in greasy wash water all year.',
 'phonetic_detail': {'start': [0,
   3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,
   28064,
   29660,
   31719,
   33360,
   33754,
   34715,
   36080,
   36326,
   37556,
   39561,
   40313,
   42059,
   43479,
   44586],
  'stop': [3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,


In [11]:
audioarray=sample["audio"]["array"]
print(len(audioarray))

46797


In [12]:
sample["audio"]["sampling_rate"]

16000

In [13]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch

In [14]:
# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [15]:
#forward sample through model to get greedily predicted transcription ids
processedsample=feature_extractor(sample["audio"]["array"],sampling_rate=sample["audio"]["sampling_rate"], return_tensors="pt")

In [16]:
processedsample

{'input_values': tensor([[ 0.0046, -0.0026,  0.0082,  ..., -0.0026, -0.0170, -0.0278]])}

In [17]:
input_values = processedsample.input_values
logits = model(input_values).logits[0]

In [18]:
logits.shape

torch.Size([145, 32])

In [19]:
pred_ids = torch.argmax(logits, axis=-1)
pred_ids

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 11,  0,  5,  0,  0,
         4,  4, 11,  0,  0,  7,  0,  0, 14,  0,  4,  4, 22,  0,  0,  8, 16,  0,
        13,  4,  4, 14,  0,  0,  0, 16,  0, 19,  0, 26,  0,  0,  4,  4,  0, 12,
         0,  0,  0,  0,  0, 16,  0,  0, 23,  0,  4,  4,  7,  9,  0, 14,  4,  4,
        21,  0, 13,  0,  5,  0,  0,  7,  0, 12,  0,  0,  0,  0,  0, 22,  0,  4,
         4, 18,  0,  0,  0,  0,  7,  0,  0, 12,  0, 11,  0,  0,  0, 18,  0,  0,
         0,  7,  0,  0,  6,  5,  5, 13,  4,  4,  4,  7, 15,  0,  0,  0, 15,  4,
         4,  4,  4, 22,  0,  0,  0,  5,  7,  7, 13,  0,  4,  4,  4,  4,  0,  0,
         0])

In [20]:
pred_ids.shape

torch.Size([145])

In [21]:
# retrieve word stamps (analogous commands for `output_char_offsets`)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)

In [22]:
# compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

In [23]:
word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in outputs.word_offsets
]

In [24]:
word_offsets[:3]

[{'word': 'SHE', 'start_time': 0.22, 'end_time': 0.32},
 {'word': 'HAD', 'start_time': 0.4, 'end_time': 0.54},
 {'word': 'YOUR', 'start_time': 0.6, 'end_time': 0.74}]

In [220]:
sample

{'file': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
 'audio': {'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
  'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
         -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]),
  'sampling_rate': 16000},
 'text': 'She had your dark suit in greasy wash water all year.',
 'phonetic_detail': {'start': [0,
   3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,
   28064,
   29660,
   31719,
   33360,
   33754,
   34715,
   36080,
   36326,
   37556,
   39561,
   40313,
   42059,
   43479,
   44586],
  'stop': [3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,


In [221]:
from transformers import AutoProcessor, Wav2Vec2ForCTC
# transcribe speech
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# audio file is decoded on the fly
inputs = processor(sample["audio"]["array"], sampling_rate=sample["audio"]["sampling_rate"], return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [222]:
inputs.input_values.shape

torch.Size([1, 46797])

In [223]:
transcription = processor.decode(pred_ids) #transcription = processor.batch_decode(pred_ids)
transcription

'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER ALL YEAR'

In [232]:
sample["text"]

'She had your dark suit in greasy wash water all year.'

In [233]:
#convert the text to upper case to match with the prediction output
inputs["labels"] = processor(text=sample["text"].upper(), return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

73.71

In [226]:
wer_metric = load_metric("wer")

In [234]:
transcription

'SHE HAD YOUR DUCK SUP AND GREASY WASHWATER ALL YEAR'

In [235]:
sample["text"]

'She had your dark suit in greasy wash water all year.'

In [236]:
wer=wer_metric.compute(predictions=[transcription], references=[sample["text"].upper()])
wer

0.5454545454545454

# Fine-tuning Wav2Vec2 for English ASR

In [237]:
from datasets import DatasetDict
raw_datasets = DatasetDict()
raw_datasets["train"] = dataset_split["train"]
validation_split_percentage = 1
num_validation_samples = raw_datasets["train"].num_rows * validation_split_percentage // 100

valkeyname="test"
raw_datasets[valkeyname] = raw_datasets["train"].select(range(num_validation_samples))
raw_datasets["train"] = raw_datasets["train"].select(range(num_validation_samples, raw_datasets["train"].num_rows))

In [238]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4574
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 46
    })
})

In [239]:
raw_datasets = raw_datasets.remove_columns(["phonetic_detail", "word_detail", 'dialect_region', 'sentence_type', 'speaker_id', 'id'])

In [127]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4574
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 46
    })
})

In [240]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [241]:
show_random_elements(raw_datasets["train"].remove_columns(["file", 'audio']), num_examples=20)

,text
0,Irish youngsters eat fresh kippers for breakfast.
1,She had your dark suit in greasy wash water all year.
2,An official deadline cannot be postponed.
3,Resolved that the anti-slavery sentiment is becoming ripe for resolute action.
4,Another put sex on a dollars-and-cents basis.
5,"I took her word for it, but is she really going with you?"
6,Westchester is a county in New York.
7,A bayonet hung in a belt scabbard.
8,The toddler found a clamshell near the camp site.
9,Most of our aid will go to those nearing self-sufficiency.


In [242]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [243]:
raw_datasets = raw_datasets.map(remove_special_characters)

In [244]:
show_random_elements(raw_datasets["train"].remove_columns(["file", "audio"]))

,text
0,don't ask me to carry an oily rag like that
1,the previous speaker presented ambiguous results
2,this process is especially difficult since gyro drifting is typically random
3,i took her word for it but is she really going with you
4,your leg muscles and back muscles feel weary
5,a lone star shone in the early evening sky
6,she found herself able to sing any role and any song which struck her fancy
7,evidence that other sources of financing are unavailable must be provided
8,cliff's display was misplaced on the screen
9,cliff's display was misplaced on the screen


In [245]:
import transformers
from transformers import (
    AdamW,
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices

In [246]:
model_name_or_path="facebook/wav2vec2-base-960h" #"facebook/wav2vec2-large-lv60" #"facebook/wav2vec2-base-960h" #"patrickvonplaten/wav2vec2-base-v2" "facebook/wav2vec2-base"

In [247]:
#option1
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

#option2
# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [136]:
feature_extractor.sampling_rate

16000

In [137]:
feature_extractor.do_normalize

True

In [138]:
from transformers import Wav2Vec2CTCTokenizer
#option1
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

#option2
# tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [140]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [248]:
import datasets
# make sure that dataset decodes audio with correct sampling rate
raw_datasets = raw_datasets.cast_column(
    "audio", datasets.features.Audio(sampling_rate=feature_extractor.sampling_rate)
)

In [249]:
raw_datasets["train"][0]

{'file': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FETB0\\SX248.WAV',
 'audio': {'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FETB0\\SX248.WAV',
  'array': array([ 9.15527344e-05,  6.10351562e-05,  3.05175781e-05, ...,
          3.05175781e-05, -3.05175781e-05, -9.15527344e-05]),
  'sampling_rate': 16000},
 'text': 'reading in poor light gives you eyestrain'}

In [143]:
raw_datasets["train"][0]["audio"]

{'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FETB0\\SX248.WAV',
 'array': array([ 9.15527344e-05,  6.10351562e-05,  3.05175781e-05, ...,
         3.05175781e-05, -3.05175781e-05, -9.15527344e-05]),
 'sampling_rate': 16000}

In [144]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(raw_datasets["train"]))

print(raw_datasets["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(raw_datasets["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

will you please describe the idiotic predicament


In [145]:
print("Input array shape:", np.asarray(raw_datasets["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", raw_datasets["train"][rand_int]["audio"]["sampling_rate"])

Input array shape: (44544,)
Sampling rate: 16000


In [171]:
raw_datasets["train"][0]['audio']

{'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FETB0\\SX248.WAV',
 'array': array([ 9.15527344e-05,  6.10351562e-05,  3.05175781e-05, ...,
         3.05175781e-05, -3.05175781e-05, -9.15527344e-05]),
 'sampling_rate': 16000}

In [170]:
raw_datasets["train"][0]['text']

'reading in poor light gives you eyestrain'

In [188]:
#with processor.as_target_processor():
result=tokenizer(raw_datasets["train"][0]['text'])
print(result)

{'input_ids': [3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [189]:
tokenizer

Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

This tokenizer does not work for the TIMIT dataset, change a new tokenizer

In [250]:
from transformers import Wav2Vec2CTCTokenizer

newtokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [192]:
newtokenizer

Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True)

In [251]:
result=newtokenizer(raw_datasets["train"][0]['text'])
print(result)

{'input_ids': [9, 0, 19, 25, 21, 4, 6, 14, 21, 4, 14, 5, 15, 15, 9, 14, 2, 21, 6, 22, 23, 14, 6, 21, 11, 0, 1, 14, 18, 15, 12, 14, 0, 18, 0, 1, 23, 9, 19, 21, 4], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [252]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=newtokenizer)

In [253]:
# set max & min audio length in number of samples
max_duration_in_seconds = 5 
min_duration_in_seconds = 3   
max_length = int(max_duration_in_seconds * feature_extractor.sampling_rate) #80000
min_length = int(min_duration_in_seconds * feature_extractor.sampling_rate) #48000

In [254]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True)

In [261]:
# def prepare_dataset(batch):
#     sample = batch["audio"]

#     inputs = feature_extractor(
#         sample["array"], sampling_rate=sample["sampling_rate"], max_length=max_length, truncation=True
#     )
#     batch["input_values"] = inputs.input_values[0]
#     batch["input_length"] = len(inputs.input_values[0])

#     return batch

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor(): #redirected to Wav2Vec2CTCTokenizer
        #add .upper() for the text
        result = processor(batch["text"])
        #print(result)
        batch["labels"] = result.input_ids
    return batch

When wrapping the processor into the as_target_processor context, however, the same method is redirected to Wav2Vec2CTCTokenizer's call method.

In [262]:
raw_datasets.column_names

{'train': ['file', 'audio', 'text'], 'test': ['file', 'audio', 'text']}

In [263]:
cache_file_names = None
vectorized_datasets = raw_datasets.map(
        prepare_dataset,
        num_proc=1,
        remove_columns=raw_datasets["train"].column_names,
        cache_file_names=cache_file_names,
    )

In [201]:
vectorized_datasets["train"]

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 4574
})

In [259]:
vectorized_datasets["train"][0]['input_length']

42189

In [264]:
processor.decode(vectorized_datasets["train"][0]['labels'])

'reading in por light gives you eyestrain'

In [265]:
#filter all sequences that are longer than 4 seconds out of the training dataset.
if max_length > 0.0:
    vectorized_datasets = vectorized_datasets.filter(
        lambda x: x < max_length,
        num_proc=1,
        input_columns=["input_length"],
    )

In [266]:
vectorized_datasets['train']

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 4455
})

In [267]:
vectorized_datasets['train'][10]['input_length']

46183

In [268]:
processor.decode(vectorized_datasets['train'][10]['labels'])

'we saw eight tiny icicles below our rof'

In [269]:
vectorized_datasets = vectorized_datasets.remove_columns("input_length")

In [209]:
vectorized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 4455
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 44
    })
})

Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. E.g., a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator

In [270]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

#ref from https://github.com/huggingface/transformers/blob/main/examples/research_projects/wav2vec2/run_asr.py
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [271]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [36]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB 8.6 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.6 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 13.0 MB/s eta 0:00:00


In [212]:
wer_metric = load_metric("wer")

In [274]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    newlabel_str=[]
    for label in label_str:
        newlabel_str.append(label.upper())

    wer = wer_metric.compute(predictions=pred_str, references=newlabel_str)

    return {"wer": wer}

In [272]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    model_name_or_path,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [273]:
model.freeze_feature_extractor()

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
#test compute_metrics


In [275]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./output/wav2vec2-finetune-asr-timit",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [217]:
valkeyname

'test'

In [276]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets[valkeyname],
    tokenizer=processor.feature_extractor,
)

In [277]:
trainer.train()

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,5.317600,3.019619,1.000000
1000,2.830000,0.530265,1.010724
1500,0.698900,0.224900,1.002681
2000,0.476400,0.143653,1.010724
2500,0.309800,0.202433,1.005362
3000,0.289600,0.133042,1.002681
3500,0.336100,0.139767,1.002681
4000,0.263000,0.130537,1.005362
4500,0.237100,0.132002,1.010724
5000,0.193500,0.132193,1.005362


c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_proc

TrainOutput(global_step=16710, training_loss=0.40843746258354985, metrics={'train_runtime': 6756.8041, 'train_samples_per_second': 19.78, 'train_steps_per_second': 2.473, 'total_flos': 3.6870105087521736e+18, 'train_loss': 0.40843746258354985, 'epoch': 30.0})

: 

trainer.push_to_hub()

## Evaluate

In [64]:
raw_datasets["test"]

Dataset({
    features: ['file', 'audio', 'text'],
    num_rows: 46
})

In [65]:
#facebook/wav2vec2-large-lv60
from transformers import AutoProcessor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

sampling_rate = raw_datasets["test"].features["audio"].sampling_rate

In [66]:
sampling_rate

16000

In [67]:
eval_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
eval_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [69]:
dataset['train'][0]

{'file': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
 'audio': {'path': 'D:\\GitHub\\datasets\\timit\\TIMIT\\train\\DR1\\FCJF0\\SA1.WAV',
  'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
         -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]),
  'sampling_rate': 16000},
 'text': 'She had your dark suit in greasy wash water all year.',
 'phonetic_detail': {'start': [0,
   3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,
   28064,
   29660,
   31719,
   33360,
   33754,
   34715,
   36080,
   36326,
   37556,
   39561,
   40313,
   42059,
   43479,
   44586],
  'stop': [3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,


In [71]:
eval_model=eval_model.to('cuda')

In [72]:
# audio file is decoded on the fly
inputs = eval_processor(dataset["test"][0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
inputs=inputs.to('cuda')
with torch.no_grad():
    logits = eval_model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

In [73]:
pred_str = eval_processor.batch_decode(predicted_ids)[0]
pred_str
#text = eval_processor.decode(dataset[0]["text"], group_tokens=False)

'SHE HAD YOUR DARK SUIT IN GREASY WASHWATER ALL YEAR'

In [74]:
dataset["test"][0]["text"]

'She had your dark suit in greasy wash water all year.'

In [75]:
# transcribe speech
transcription = eval_processor.batch_decode(predicted_ids)
transcription[0]


'SHE HAD YOUR DARK SUIT IN GREASY WASHWATER ALL YEAR'

In [76]:
inputs["labels"] = eval_processor(text=dataset["test"][0]["text"], return_tensors="pt").input_ids

In [77]:
inputs

{'input_values': tensor([[0.0037, 0.0062, 0.0024,  ..., 0.0100, 0.0126, 0.0088]],
       device='cuda:0'), 'labels': tensor([[12,  3,  3,  4,  3,  3,  3,  4,  3,  3,  3,  3,  4,  3,  3,  3,  3,  4,
          3,  3,  3,  3,  4,  3,  3,  4,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,
          3,  4,  3,  3,  3,  3,  3,  4,  3,  3,  3,  4,  3,  3,  3,  3,  3]])}

In [78]:
# compute loss, inputs as labels and input_values
loss = eval_model(**inputs).loss
round(loss.item(), 2)

1356.02

## Evaluate own trained model

In [79]:
mymodel = Wav2Vec2ForCTC.from_pretrained(".\output\wav2vec2-finetune-asr-timit\checkpoint-16500")

In [80]:
mymodel=mymodel.cuda()

In [81]:
myprocessor = Wav2Vec2Processor.from_pretrained(".\output\wav2vec2-finetune-asr-timit\checkpoint-16500")

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for '.\output\wav2vec2-finetune-asr-timit\checkpoint-16500'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '.\output\wav2vec2-finetune-asr-timit\checkpoint-16500' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [82]:
tokenizer.save_pretrained(".\output\wav2vec2-finetune-asr-timit\checkpoint-16500")

('.\\output\\wav2vec2-finetune-asr-timit\\checkpoint-16500\\tokenizer_config.json',
 '.\\output\\wav2vec2-finetune-asr-timit\\checkpoint-16500\\special_tokens_map.json',
 '.\\output\\wav2vec2-finetune-asr-timit\\checkpoint-16500\\vocab.json',
 '.\\output\\wav2vec2-finetune-asr-timit\\checkpoint-16500\\added_tokens.json')

In [83]:
myprocessor = Wav2Vec2Processor.from_pretrained(".\output\wav2vec2-finetune-asr-timit\checkpoint-16500")

In [84]:
myprocessor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='.\output\wav2vec2-finetune-asr-timit\checkpoint-16500', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [85]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [86]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = mymodel(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = myprocessor.batch_decode(pred_ids)[0]
  batch["text"] = myprocessor.decode(batch["labels"], group_tokens=False)
  
  return batch

we will make use of the map(...) function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary "results".

Note: we evaluate the test data set with batch_size=1 on purpose due to this issue. Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [87]:
vectorized_datasets["test"]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 44
})

In [98]:
vectorized_datasets["test"][0]

{'input_values': [0.004556330386549234,
  -0.0026364438235759735,
  0.008152717724442482,
  0.0009599433979019523,
  0.0009599433979019523,
  0.011749105527997017,
  0.0009599433979019523,
  0.004556330386549234,
  -0.009829217568039894,
  -0.006232831161469221,
  0.008152717724442482,
  0.004556330386549234,
  0.0009599433979019523,
  0.0009599433979019523,
  0.011749105527997017,
  0.018941879272460938,
  0.018941879272460938,
  0.018941879272460938,
  0.018941879272460938,
  0.018941879272460938,
  0.015345491468906403,
  0.011749105527997017,
  0.018941879272460938,
  0.018941879272460938,
  0.018941879272460938,
  0.018941879272460938,
  0.015345491468906403,
  0.004556330386549234,
  0.018941879272460938,
  0.0009599433979019523,
  0.008152717724442482,
  0.022538265213370323,
  0.015345491468906403,
  0.018941879272460938,
  0.015345491468906403,
  0.022538265213370323,
  0.015345491468906403,
  0.008152717724442482,
  0.008152717724442482,
  -0.009829217568039894,
  -0.00263644

In [99]:
vectorized_datasets["test"][0]['labels']

[3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3]

In [92]:
labelresult=myprocessor.decode(vectorized_datasets["test"][0]['labels'], group_tokens=False) 

In [93]:
labelresult

'<unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk>'

In [94]:
results = vectorized_datasets["test"].map(map_to_result, remove_columns=vectorized_datasets["test"].column_names)

Parameter 'function'=<function map_to_result at 0x0000014C3972D1B0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 44/44 [00:04<00:00, 10.95 examples/s]


compute the overall WER

In [95]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.971


In [96]:
results

Dataset({
    features: ['pred_str', 'text'],
    num_rows: 44
})

In [97]:
show_random_elements(results)

,pred_str,text
0,<unk>,<unk><unk><unk>'<unk> <unk><unk><unk> <unk><unk> <unk><unk> <unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk>
1,<unk>,<unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk>
2,<unk>,<unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk>
3,<unk>,<unk><unk> <unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk> <unk> <unk><unk><unk><unk><unk>
4,<unk>,<unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
5,<unk>,<unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk>
6,<unk>,<unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk> <unk><unk> <unk> <unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk>
7,<unk>,<unk><unk><unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk> <unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk><unk><unk>
8,<unk>,<unk><unk><unk> <unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk> <unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk>
9,<unk>,<unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk> <unk><unk> <unk><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk>


In [212]:
with torch.no_grad():
  logits = mymodel(torch.tensor(vectorized_datasets["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> I I T <pad> | | W W A <pad> S S | | <pad> I I M <pad> <pad> <pad> P <pad> <pad> <pad> <pad> O S <pad> <pad> <pad> S S <pad> <pad> I I <pad> B <pad> L L L E | | <pad> <pad> T <pad> <pad> O O | | R R <pad> E <pad> <pad> <pad> S <pad> <pad> <pad> <pad> I I S S <pad> T T <pad> | | | <pad> <pad> I <pad> N <pad> | | | <pad> E E <pad> X <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> I I S S <pad> <pad> T T <pad> <pad> <pad> <pad> <pad> E N N <pad> <pad> C E E <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

# Custom Training loop

In [213]:
from torch.utils.data.dataloader import DataLoader
batch_size = 8
train_dataloader = DataLoader(
    vectorized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)


In [214]:
# Optimizer
learning_rate=5e-5
adam_beta1=0.9
adam_beta2=0.999
adam_epsilon=1e-8
optimizer = AdamW(
    list(model.parameters()),
    lr=learning_rate,
    betas=[adam_beta1, adam_beta2],
    eps=adam_epsilon,
)

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [215]:
import math
gradient_accumulation_steps =1
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

In [216]:
num_update_steps_per_epoch

242

In [217]:
num_train_epochs = 6
max_train_steps = num_train_epochs * num_update_steps_per_epoch

In [218]:
num_warmup_steps = 10
lr_scheduler = get_scheduler(
        name="linear", #["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_train_steps,
    )

In [220]:
#recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

In [221]:
total_batch_size = batch_size * gradient_accumulation_steps

In [222]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [223]:
gpuid=0
if torch.cuda.is_available():
    device = torch.device('cuda:'+str(gpuid))  # CUDA GPU 0
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [228]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(max_train_steps))
completed_steps = 0
starting_epoch = 0
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    losses=[]
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        #backward
        loss.backward()

        if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1
        
    print(f"epoch {epoch}, loss: {loss}")

 17%|█▋        | 242/1452 [01:51<08:15,  2.44it/s]

epoch 0, loss: 0.016273705288767815


 33%|███▎      | 484/1452 [03:43<06:07,  2.64it/s]

epoch 1, loss: 0.018542783334851265


 50%|█████     | 726/1452 [05:34<04:52,  2.48it/s]

...

epoch 3, loss: 0.015551198273897171


 83%|████████▎ | 1210/1452 [09:18<01:36,  2.51it/s]

epoch 4, loss: 0.03556506335735321


100%|██████████| 1452/1452 [11:09<00:00,  2.55it/s]

epoch 5, loss: 0.04022493213415146


In [234]:
eval_dataloader = DataLoader(
    vectorized_datasets[valkeyname], collate_fn=data_collator, batch_size=1
)

In [247]:
batch=next(iter(eval_dataloader))

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [249]:
batch["labels"].shape

torch.Size([1, 40])

In [250]:
batch["input_values"].shape

torch.Size([1, 47360])

In [275]:
def evaluate_dataset(model, processor, eval_dataloader, device):
    # Evaluation
    totallen = len(eval_dataloader)
    print("Total evaluation length:", totallen)
    model.eval()
    predictions=[]
    references=[]
    for batch in tqdm(eval_dataloader):
        #batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            #outputs = model(**batch)
            print("input shape:", batch["input_values"].shape)
            input_values = torch.tensor(batch["input_values"], device=device)#.unsqueeze(0)
            logits = model(input_values).logits
        #logits = outputs.logits
        print(logits.shape)
        predicted_ids = torch.argmax(logits[0], dim=-1)
        # transcribe speech
        print("predicted_ids shape", predicted_ids.shape)
        #print(predicted_ids)
        pred_str = processor.decode(predicted_ids)
        #batch_decode results do not perform tokenization?
        predictions.append(pred_str)
        text_str = processor.decode(batch["labels"][0], group_tokens=False)
        references.append(text_str)
        print("pred_str", pred_str)
        print("text_str", text_str)
    werscore=wer_metric.compute(predictions=predictions, references=references)
    print("Test WER: {:.3f}".format(werscore))

In [243]:
len(eval_dataloader)

10

In [276]:
evaluate_dataset(mymodel, processor, eval_dataloader, device)

Total evaluation length: 10


input shape: torch.Size([1, 47360])
torch.Size([1, 147, 32])
predicted_ids shape torch.Size([147])
pred_str IT WAS IMPOSSIBLE TO RESIST IN EXISTENCE
text_str IT WAS IMPOSSIBLE TO RESIST AN EXISTENCE
input shape: torch.Size([1, 64320])
torch.Size([1, 200, 32])
predicted_ids shape torch.Size([200])


c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
C:\Users\lkk68\AppData\Local\Temp\ipykernel_15572\2957877289.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(batch["input_values"], device=device)#.unsqueeze(0)



pred_str I HAVE TRAVELLED A GOOD DEAL AND SEEN MUCH GRANDER THINGS
text_str I HAVE TRAVELLED A GOOD DEAL AND SEEN MUCH GRANDER THINGS
input shape: torch.Size([1, 52640])
torch.Size([1, 164, 32])
predicted_ids shape torch.Size([164])
pred_str IN THE COURSE OF THE DAY I RECEIVED THIS NOTE
text_str IN THE COURSE OF THE DAY I RECEIVED THIS NOTE
input shape: torch.Size([1, 44960])
torch.Size([1, 140, 32])
predicted_ids shape torch.Size([140])
pred_str BE AT PRUDENCE'S TONIGHT AT EIGHT
text_str BE AT PRUDENCE'S TO NIGHT AT EIGHT


 40%|████      | 4/10 [00:00<00:00,  9.76it/s]

input shape: torch.Size([1, 66320])
torch.Size([1, 207, 32])
predicted_ids shape torch.Size([207])
pred_str NO AT POINT DEJOUR WHERE WE HAD DINNER THE DUKE AND I
text_str NO AT POINT DU JOUR WHERE WE HAD DINNER THE DUKE AND I
input shape: torch.Size([1, 76320])
torch.Size([1, 238, 32])
predicted_ids shape torch.Size([238])
pred_str I HAPPENED TO DODGE A BALL FIRED FROM THE OTHER SIDE AND IT WENT THROUGH THE GLASS WHAT
text_str I HAPPENED TO DODGE A BALL FIRED FROM THE OTHER SIDE AND IT WENT THROUGH THE GLASS WHAT
input shape: torch.Size([1, 61840])
torch.Size([1, 193, 32])
predicted_ids shape torch.Size([193])


pred_str ALTHOUGH THE LATTER HAD NOT BEEN A RESIDENT THERE MUCH MORE THAN THREE YEARS
text_str ALTHOUGH THE LATTER HAD NOT BEEN A RESIDENT THERE MUCH MORE THAN THREE YEARS
input shape: torch.Size([1, 44560])
torch.Size([1, 139, 32])
predicted_ids shape torch.Size([139])
pred_str AS THE FOUR CHUMS WENT AWAY JERRY CHUCKLED
text_str AS THE FOUR CHUMS WENT AWAY JERRY CHUCKLED
input shape: torch.Size([1, 52000])
torch.Size([1, 162, 32])
predicted_ids shape torch.Size([162])
pred_str OH NO IT ISN'T SO BAD AS THAT HE WAS ASSURED
text_str OH NO IT ISN'T SO BAD AS THAT HE WAS ASSURED
input shape: torch.Size([1, 54240])
torch.Size([1, 169, 32])
predicted_ids shape torch.Size([169])


100%|██████████| 10/10 [00:00<00:00, 11.31it/s]

pred_str WELL SO LONG BOYS AND WE ALL WISH YOU SUCCESS
text_str WELL SO LONG BOYS AND WE ALL WISH YOU SUCCESS
Test WER: 0.046
